### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

# Goal-oriented bot in DeepPavlov

The tutor is focused on building a goal-oriented dialogue system:

0. [Data preparation](#0.-Data-Preparation)
1. [Train bot](#1.-Train-bot)

An example of the final model served as a telegram bot is:

![gobot_simple_example.png](img/gobot_simple_example.png)

## 0. Data Preparation

The tutor's dialogue system will be on the domain of personal assistant. We generated 10 dialogues to train and validate out bot.

See below a small chunk of the data.

In [50]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader


class AssistantDatasetReader(SimpleDSTC2DatasetReader):
    
    url = "http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz"
    
    @staticmethod
    def _data_fname(datatype):
        assert datatype in ('val', 'trn', 'tst'), "wrong datatype name"
        return f"assistant-{datatype}.json"

In [51]:
data = AssistantDatasetReader().read('assistant_data')

2019-09-06 17:04:11.586 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [PosixPath('assistant_data/assistant-val.json'), PosixPath('assistant_data/assistant-tst.json')]]
I0906 17:04:11.586643 139767249307264 dstc2_reader.py:269] [PosixPath('assistant_data/assistant-val.json'), PosixPath('assistant_data/assistant-tst.json')]]
2019-09-06 17:04:11.587 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 270: [downloading data from http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz to assistant_data]
I0906 17:04:11.587791 139767249307264 dstc2_reader.py:270] [downloading data from http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz to assistant_data]
2019-09-06 17:04:12.96 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz to assistant_data/tutor_assistant_data.tar.gz
I0906 17:04:12.096202 139767249307264 utils.py:63] Downloadi

In [52]:
!ls assistant_data

assistant-templates.txt  assistant-tst.json
assistant-trn.json	 assistant-val.json


The training/validation/test data is stored in json files (`assistant-trn.json`, `assistant-val.json` and `assistant-tst.json`):

In [53]:
!head -n 31 assistant_data/assistant-trn.json

[
  [
    {
      "speaker": 1,
      "text": "hi"
    },
    {
      "speaker": 2,
      "text": "Hello, what is the weather today?",
      "act": "welcome_msg"
    },
    {
      "speaker": 1,
      "text": "Quite sunny outside"
    },
    {
      "speaker": 2,
      "text": "Then you should cycle!",
      "act": "suggest_cycling"
    },
    {
      "speaker": 1,
      "text": "Thanks! Great idea"
    },
    {
      "speaker": 2,
      "text": "You are welcome! Bye!",
      "act": "good_bye"
    }
  ],
  [


In [54]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed dialogs:

In [55]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

print("\n-----------------")    
print(f"{len(iterator.get_instances('train')[0])} dialog(s) in train.")
print(f"{len(iterator.get_instances('valid')[0])} dialog(s) in valid.")
print(f"{len(iterator.get_instances('test')[0])} dialog(s) in test.")

User utterances:
----------------

[   {'prev_resp_act': None, 'text': 'hey'},
    {'prev_resp_act': 'welcome_msg', 'text': 'very dark and murky'},
    {'prev_resp_act': 'suggest_tea', 'text': 'tea is not funny'}]

System responses:
-----------------

[   {'act': 'welcome_msg', 'text': 'Hello, what is the weather today?'},
    {'act': 'suggest_tea', 'text': 'Then you should try hot chinese tea!'},
    {   'act': 'bad_bye',
        'text': "That's a pity! Next time maybe. Have a good day!"}]

-----------------
8 dialog(s) in train.
1 dialog(s) in valid.
1 dialog(s) in test.


## 1. Train bot

Let's train the main module: dialogue policy network.

&nbsp;
![gobot_simple_policy.png](img/gobot_simple_policy.png)
&nbsp;

Policy network decides which action the system should take on each turn of a dialogue: should it say goodbye, request user's location or say thanks.

The policy network is a recurrent neural network (recurrent over utterances represented as bags of words) and a dense layer with softmax function on top. The network classifies user utterance into one of predefined system actions.

&nbsp;
![gobot_simple_templates.png](img/gobot_simple_templates.png)
&nbsp;

All actions available for the system should be listed in a `assistant-templates.txt` file. Each action should be associated with a string of the corresponding system response.

Templates should be in the format `<act>TAB<template>`, where `<act>` is a dialogue action and `<template>` is the corresponding response.

In [29]:
!head -n 10 assistant_data/assistant-templates.txt

welcome_msg	Hello, what is the weather today?
suggest_tea	Then you should try hot chinese tea!
suggest_cycling	Then you should cycle!
good_bye	You are welcome! Bye!
bad_bye	That's a pity! Next time maybe. Have a good day!


So, actions are actually classes we classify over. And `assistant-templates.txt` contains the set of classes.

To train the dialogue policy network for classification task you need action label for each system utterance in training dialogues. The dataset contains `"act"` dictionary key that contains action associated with current response.

The cell below provides an example of training data for the policy network.

In [56]:
!head -n 31 assistant_data/assistant-trn.json

[
  [
    {
      "speaker": 1,
      "text": "hi"
    },
    {
      "speaker": 2,
      "text": "Hello, what is the weather today?",
      "act": "welcome_msg"
    },
    {
      "speaker": 1,
      "text": "Quite sunny outside"
    },
    {
      "speaker": 2,
      "text": "Then you should cycle!",
      "act": "suggest_cycling"
    },
    {
      "speaker": 1,
      "text": "Thanks! Great idea"
    },
    {
      "speaker": 2,
      "text": "You are welcome! Bye!",
      "act": "good_bye"
    }
  ],
  [


Let's **construct the final pipeline** of a dialogue system.

We take [default DSTC2 bot config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_dstc2.json) ([more configs](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot) are available) and change sections responsible for 
- templates,
- embeddings,
- data and model load/save paths.

In [57]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2_minimal)

**Configure** bot to use **templates**:

In [58]:
gobot_config['chainer']['pipe'][-1]['template_path'] = 'assistant_data/assistant-templates.txt'

You should use pretrained **glove embeddings** (trainable embeddings are default):

In [59]:
from deeppavlov.download import download_resource

download_resource(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
                  dest_paths=['assistant_bot/'])

2019-09-06 17:12:33.155 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt download because of matching hashes
I0906 17:12:33.155169 139767249307264 download.py:117] Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt download because of matching hashes


In [60]:
gobot_config['chainer']['pipe'][-1]['embedder'] = {
    "class_name": "glove",
    "load_path": "assistant_bot/glove.6B.100d.txt"
}

Specify train/valid/test **data path** and **path to save** the final bot model:

In [61]:
gobot_config['dataset_reader']['class_name'] = '__main__:AssistantDatasetReader'
gobot_config['metadata']['variables']['DATA_PATH'] = 'assistant_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'assistant_bot'

The whole dialogue system pipeline looks like this:
    
![gobot_simple_pipeline.png](img/gobot_simple_pipeline.png)

##### Training policy network

In [62]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 4 # set batch size
gobot_config['train']['max_batches'] = 30 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 30 # evaluate on full 'valid' split every 30 epochs
gobot_config['train']['log_every_n_batches'] = 5 # evaluate on full 'train' split every 5 batches

train_model(gobot_config);

2019-09-06 18:00:43.348 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-trn.json]
I0906 18:00:43.348532 139767249307264 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-trn.json]
2019-09-06 18:00:43.349 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-val.json]
I0906 18:00:43.349636 139767249307264 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-val.json]
2019-09-06 18:00:43.350 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-tst.json]
I0906 18:00:43.350584 139767249307264 dstc2_reader.py:290] [loading dialogs fr

{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01", "epochs_done": 2, "batches_seen": 5, "train_examples_seen": 20, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.08294930309057236}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01", "epochs_done": 4, "batches_seen": 10, "train_examples_seen": 40, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.0416302278637886}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01", "epochs_done": 7, "batches_seen": 15, "train_examples_seen": 60, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.03172624669969082}}
{"train": {"eval_examples_count": 8, "metrics": {"per_it

2019-09-06 18:01:09.704 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 170: Did not improve on the per_item_dialog_accuracy of 1.0
I0906 18:01:09.704594 139767249307264 nn_trainer.py:170] Did not improve on the per_item_dialog_accuracy of 1.0


{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:02", "epochs_done": 12, "batches_seen": 25, "train_examples_seen": 100, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.027604592964053154}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:02", "epochs_done": 14, "batches_seen": 30, "train_examples_seen": 120, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.02618459761142731}}
{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:02", "epochs_done": 14, "batches_seen": 30, "train_examples_seen": 120, "impatience": 1, "patience_limit": 10}}


2019-09-06 18:01:09.752 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/assistant_bot/word.dict]
I0906 18:01:09.752158 139767249307264 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/assistant_bot/word.dict]
2019-09-06 18:01:09.754 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/home/vimary/code-projects/Pilot/examples/assistant_bot/glove.6B.100d.txt`]
I0906 18:01:09.754283 139767249307264 glove_embedder.py:52] [loading GloVe embeddings from `/home/vimary/code-projects/Pilot/examples/assistant_bot/glove.6B.100d.txt`]
/home/vimary/dp_env/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the

{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01"}}


2019-09-06 18:01:59.966 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 161: [loading templates from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-templates.txt]
I0906 18:01:59.966386 139767249307264 network.py:161] [loading templates from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-templates.txt]
2019-09-06 18:01:59.967 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: 5 templates loaded.
I0906 18:01:59.967494 139767249307264 network.py:164] 5 templates loaded.
2019-09-06 18:01:59.968 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 210: Calculated input size for `GoalOrientedBotNetwork` is 129
I0906 18:01:59.968229 139767249307264 network.py:210] Calculated input size for `GoalOrientedBotNetwork` is 129
2019-09-06 18:02:00.482 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
I0906 18:02:00.482819 139767249307264 network.py:248] [initial

Training on the dataset takes up to 5 minutes depending on gpu/cpu.

See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of the training process.

##### Metric scores on valid&test

Calculating **accuracy** of trained bot: whether predicted system responses match true responses (full string match).

In [37]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2019-09-06 15:04:24.832 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-trn.json]
I0906 15:04:24.832241 139767249307264 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-trn.json]
2019-09-06 15:04:24.833 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-val.json]
I0906 15:04:24.833665 139767249307264 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-val.json]
2019-09-06 15:04:24.834 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/assistant_data/assistant-tst.json]
I0906 15:04:24.834707 139767249307264 dstc2_reader.py:290] [loading dialogs fr

{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01"}}


With settings of `max_batches=30`, system response accuracy on valid and test equals `1.0` (with valid and test dialogues consisting of 1 dialogue each).

##### Chatting with bot

In [38]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-09-06 15:04:50.566 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/assistant_bot/word.dict]
I0906 15:04:50.566690 139767249307264 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/assistant_bot/word.dict]
2019-09-06 15:04:50.568 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/home/vimary/code-projects/Pilot/examples/assistant_bot/glove.6B.100d.txt`]
I0906 15:04:50.568121 139767249307264 glove_embedder.py:52] [loading GloVe embeddings from `/home/vimary/code-projects/Pilot/examples/assistant_bot/glove.6B.100d.txt`]
/home/vimary/dp_env/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the

In [39]:
bot(['good evening, bot'])

['Hello, what is the weather today?']

In [40]:
bot(['the weather is clooudy and gloooomy'])

['Then you should try hot chinese tea!']

In [41]:
bot(["nice idea, thanks!"])

['You are welcome! Bye!']

In [42]:
bot.reset()

In [43]:
bot(['hi bot'])

['Hello, what is the weather today?']

In [44]:
bot(["looks ok, the sun is bright and yesterday's rain stopped already"])

['Then you should cycle!']

In [45]:
bot(['i dont wanna'])

["That's a pity! Next time maybe. Have a good day!"]